In [21]:
import awswrangler as wr

from data_sci_toolkit.aws_tools import permission_tools 

In [22]:
SESSION = permission_tools.get_aws_session("Dev-DataScienceAdmin")

Authorized as AROAQBUNWK7XN5DTYWPLJ:rkazmerik@stellaralgo.com


In [23]:
DATASOURCE = "AwsDataCatalog"
DATABASE = "integrations_milb_avaiators"
SQL = f"""
    SELECT * FROM {DATABASE} LIMIT 1
"""
WORKGROUP = "primary"

In [24]:
df = wr.athena.read_sql_query(
    boto3_session= SESSION, 
    database= DATABASE, 
    sql= SQL, 
    workgroup= WORKGROUP)
df.shape

wr.athena.read_sql_query()

WaiterError: Waiter BucketExists failed: Max attempts exceeded. Previously accepted state: Matched expected HTTP status code: 404